### Python API tutorial using requests Package 

Application Program Interfaces, or APIs, are commonly used to retrieve data from remote websites. Sites like Reddit, Twitter, and Facebook all offer certain data through their APIs. To use an API, you make a request to a remote web server, and retrieve the data you need.

In [1]:
import requests

# Make a get request to get the latest position of the international space station from the opennotify api.
response = requests.get("http://api.open-notify.org/iss-now.json")

# Print the status code of the response.
print(response.status_code)

200


Status codes

The request we just made had a status code of 200. Status codes are returned with every request that is made to a web server. Status codes indicate information about what happened with a request. Here are some codes that are relevant to GET requests:

* 200 -- everything went okay, and the result has been returned (if any)
* 301 -- the server is redirecting you to a different endpoint. This can happen when a company switches domain names, or an endpoint name is changed.
* 401 -- the server thinks you're not authenticated. This happens when you don't send the right credentials to access an API.
* 400 -- the server thinks you made a bad request. This can happen when you don't send along the right data, among other things.
* 403 -- the resource you're trying to access is forbidden -- you don't have the right permissions to see it.
* 404 -- the resource you tried to access wasn't found on the server.

We'll now make a GET request to http://api.open-notify.org/iss-pass, an endpoint that doesn't exist, per the API documentation.

In [2]:
# get request to non existing end point
r = requests.get("http://api.open-notify.org/iss-pass")
print(r.status_code)

404


iss-pass wasn't a valid endpoint, so we got a 404 status code in response. We forgot to add .json at the end, as the API documentation states.

We'll now make a GET request to http://api.open-notify.org/iss-pass.json.



In [3]:
response = requests.get("http://api.open-notify.org/iss-pass.json")
print(response.status_code)


400


You'll see that in the last example, we got a 400 status code, which indicates a bad request. If you look at the documentation for the OpenNotify API, we see that the ISS Pass endpoint requires two parameters.
* lat -- The latitude of the location we want.
* lon -- The longitude of the location we want.

We can make a dictionary with these parameters, and then pass them into the requests.get function.

We can also do the same thing directly by adding the query parameters to the url, like this: http://api.open-notify.org/iss-pass.json?lat=40.71&lon=-74.

It's almost always preferable to setup the parameters as a dictionary, because requests takes care of some things that come up, like properly formatting the query parameters.

In [4]:
# Set up the parameters we want to pass to the API.
# This is the latitude and longitude of New York City.
parameters = {"lat": 40.71, "lon": -74}

# Make a get request with the parameters.
response = requests.get("http://api.open-notify.org/iss-pass.json", params=parameters)

# Print the content of the response (the data the server returned)
print(response.content)

# This gets the same data as the command above
response = requests.get("http://api.open-notify.org/iss-pass.json?lat=40.71&lon=-74")
print(response.content)

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1542101406, \n    "latitude": 40.71, \n    "longitude": -74.0, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 642, \n      "risetime": 1542103704\n    }, \n    {\n      "duration": 505, \n      "risetime": 1542109533\n    }, \n    {\n      "duration": 113, \n      "risetime": 1542158208\n    }, \n    {\n      "duration": 611, \n      "risetime": 1542163694\n    }, \n    {\n      "duration": 623, \n      "risetime": 1542169475\n    }\n  ]\n}\n'
b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1542101406, \n    "latitude": 40.71, \n    "longitude": -74.0, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 642, \n      "risetime": 1542103704\n    }, \n    {\n      "duration": 505, \n      "risetime": 1542109533\n    }, \n    {\n      "duration": 113, \n      "risetime": 1542158208\n    }, \n    {\n      "duration": 611, \n      "r

### Working with JSON data


JavaScript Object Notation (JSON)
use json package to work with json in python
JSON is a way to encode data structures like lists and dictionaries to strings that ensures that they are easily readable 
by machines. JSON is the primary format in which data is passed back and forth to APIs, and most API servers will send 
their responses in JSON format

The json library has two main methods:

* dumps -- Takes in a Python object, and converts it to a string.
* loads -- Takes a JSON string, and converts it to a Python object.

You can get the content of a response as a python object by using the .json() method on the response.

In [5]:
# Make the same request we did earlier, but with the coordinates of San Francisco instead.
parameters = {"lat": 37.78, "lon": -122.41}
response = requests.get("http://api.open-notify.org/iss-pass.json", params=parameters)

# Get the response data as a python object.  Verify that it's a dictionary.
data = response.json()
print(type(data))
print(data)


<class 'dict'>
{'message': 'success', 'request': {'altitude': 100, 'datetime': 1542102145, 'latitude': 37.78, 'longitude': -122.41, 'passes': 5}, 'response': [{'duration': 471, 'risetime': 1542103221}, {'duration': 532, 'risetime': 1542109058}, {'duration': 635, 'risetime': 1542114835}, {'duration': 560, 'risetime': 1542120644}, {'duration': 196, 'risetime': 1542169228}]}


### Content type

The server doesn't just send a status code and the data when it generates a response. It also sends metadata containing information on how the data was generated and how to decode it. This is stored in the response headers. In Python, we can access this with the headers property of a response object.

The headers will be shown as a dictionary. Within the headers, content-type is the most important key for now. It tells us the format of the response, and how to decode it. For the OpenNotify API, the format is JSON, which is why we could decode it with the json package earlier.



In [6]:
# Headers is a dictionary
print(response.headers)

# Get the content-type from the dictionary.
print(response.headers["content-type"])

{'Server': 'nginx/1.10.3', 'Date': 'Tue, 13 Nov 2018 10:06:48 GMT', 'Content-Type': 'application/json', 'Content-Length': '521', 'Connection': 'keep-alive', 'Via': '1.1 vegur'}
application/json


Finding the number of people in space

OpenNotify has one more API endpoint, astros.json. It tells you how many people are currently in space. 

In [7]:
# Get the response from the API endpoint.
response = requests.get("http://api.open-notify.org/astros.json")
data = response.json()

# 9 people are currently in space.
print(data["number"])
print(data)

3
{'people': [{'name': 'Sergey Prokopyev', 'craft': 'ISS'}, {'name': 'Alexander Gerst', 'craft': 'ISS'}, {'name': 'Serena Aunon-Chancellor', 'craft': 'ISS'}], 'message': 'success', 'number': 3}


Authentication with API's

In [9]:
from requests.auth import HTTPBasicAuth

response = requests.get('https://github.com/diljeet1994', auth=HTTPBasicAuth('user', 'pass'))
print(response.status_code)

new_response = requests.get('https://github.com/diljeet1994', auth=('diljeet1994', 'password'))

new_response.status_code
# Digest Authentication, OAuth 1 Authentication, OAuth 2 and OpenID Connect Authentication
# http://docs.python-requests.org/en/master/user/authentication/

200


200